<a href="https://colab.research.google.com/github/SLokesh2509/Reliable_raptors/blob/main/Heat_wave_occurrence_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report
from sklearn.impute import SimpleImputer


In [5]:
# Load data
old_dataset = pd.read_excel('Book2.xlsx')

In [6]:
#defining the districts
district=input('Enter the district:')


Enter the district:Adilabad


In [8]:
dataset=old_dataset[old_dataset['District']==district]

In [9]:
#preprocessing the data
dataset = dataset.dropna()

In [10]:
#print(dataset.loc[:,"Date"].head(10))
dataset["Date"] = pd.to_datetime(dataset["Date"])
dataset["year"] = pd.DatetimeIndex(dataset["Date"]).year
dataset["month"] = pd.DatetimeIndex(dataset["Date"]).month
dataset["day"] = pd.DatetimeIndex(dataset["Date"]).day
print(dataset.head(5))

   District          Mandal       Date  Rainfall (mm)  temp_min (⁰C)  \
0  Adilabad  Adilabad Rural 2022-04-01            0.0           22.0   
1  Adilabad  Adilabad Rural 2022-04-02            0.0           26.6   
2  Adilabad  Adilabad Rural 2022-04-03            0.0           26.9   
3  Adilabad  Adilabad Rural 2022-04-04            0.0           28.3   
4  Adilabad  Adilabad Rural 2022-04-05            0.0           28.2   

   temp_max (⁰C)  humidity_min (%)  Humidity_max (%)  wind_speed_min (Kmph)  \
0           42.6              15.7              56.2                    0.0   
1           42.0              15.7              80.8                    2.5   
2           42.2              19.8              78.5                    0.1   
3           42.2              24.8              72.0                    0.0   
4           41.0              32.1              68.6                    2.2   

   wind_speed_max (Kmph)  year  month  day  
0                   13.0  2022      4    1  
1 

In [11]:
# Define the thresholds for each parameter that would indicate the start of a heat wave
rain_threshold = 0.5 # in mm
temperature_threshold = 37 # in ⁰C
min_humidity_threshold = 30 # in %
max_humidity_threshold = 70 # in %
min_wind_speed_threshold = 10 # in Kmph
max_wind_speed_threshold = 25 # in Kmph

# Create a new column that indicates whether a heat wave occurred on each day
# A heat wave is defined as three or more consecutive days with rain below the threshold,
# minimum humidity above the threshold, maximum humidity above the threshold,
# minimum wind speed below the threshold, and maximum wind speed below the threshold
dataset["Heat Wave"] = ((dataset["Rainfall (mm)"] < rain_threshold) &
                   #(dataset["humidity_min (%)"] > min_humidity_threshold) &
                   (dataset["Humidity_max (%)"] > max_humidity_threshold) &
                   #(dataset["wind_speed_min (Kmph)"] < min_wind_speed_threshold) &
                   #(dataset["wind_speed_max (Kmph)"] < max_wind_speed_threshold) &
                   (dataset["temp_max (⁰C)"] > temperature_threshold)&
                   (dataset["Rainfall (mm)"].rolling(3).sum() >= 0.75)).astype(int)

# Create a new column that categorizes the heat waves based on their intensity
# Mild: 3-5 consecutive days with heat wave occurrence
# Moderate: 6-8 consecutive days with heat wave occurrence
# Severe: 9 or more consecutive days with heat wave occurrence
dataset["Heat Wave Intensity"] = pd.cut(dataset["Heat Wave"].groupby((dataset["Heat Wave"].diff() != 0).cumsum()).cumsum(),
                                   bins=[0, 5, 8, 1000],
                                   labels=["Mild", "Moderate", "Severe"])


In [12]:
#features = ['Rainfall (mm)','temp_min (⁰C)','temp_max (⁰C)','humidity_min (%)','Humidity_max (%)','wind_speed_min (Kmph)','wind_speed_max (Kmph)','year','month','day']
features = ['year','month','day']
print(dataset.head(5))

   District          Mandal       Date  Rainfall (mm)  temp_min (⁰C)  \
0  Adilabad  Adilabad Rural 2022-04-01            0.0           22.0   
1  Adilabad  Adilabad Rural 2022-04-02            0.0           26.6   
2  Adilabad  Adilabad Rural 2022-04-03            0.0           26.9   
3  Adilabad  Adilabad Rural 2022-04-04            0.0           28.3   
4  Adilabad  Adilabad Rural 2022-04-05            0.0           28.2   

   temp_max (⁰C)  humidity_min (%)  Humidity_max (%)  wind_speed_min (Kmph)  \
0           42.6              15.7              56.2                    0.0   
1           42.0              15.7              80.8                    2.5   
2           42.2              19.8              78.5                    0.1   
3           42.2              24.8              72.0                    0.0   
4           41.0              32.1              68.6                    2.2   

   wind_speed_max (Kmph)  year  month  day  Heat Wave Heat Wave Intensity  
0               

In [13]:
target = dataset['Heat Wave'].to_frame()
#dataset.dropna()
#print(dataset.head(10))

In [14]:
# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(dataset[features], target, test_size=0.3, random_state=42)
#print(X_train)
# Train a random forest classifier
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)

# Make predictions on test set
y_pred = rf.predict(X_test)

#report
report = classification_report(y_test, y_pred)
print(report)

# Evaluate accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy: {:.2f}%".format(accuracy*100))


<ipython-input-14-b2963a8c8819>:6: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf.fit(X_train, y_train)


              precision    recall  f1-score   support

           0       0.99      1.00      1.00      7764
           1       0.50      0.05      0.08        65

    accuracy                           0.99      7829
   macro avg       0.75      0.52      0.54      7829
weighted avg       0.99      0.99      0.99      7829

Accuracy: 99.17%


In [15]:
import pickle

# Save the trained model to disk
filename = 'rf.pkl'
with open(filename, 'wb') as file:
    pickle.dump(rf, file)

# Load the saved model from disk
with open(filename, 'rb') as file:
    loaded_model = pickle.load(file)

# Define the new input data
year=int(input('Enter the year:'))
month=int(input('Enter the Month:'))
date=int(input('Enter the date:'))
data_list = [year,month,date]

# Convert the list to a pandas dataframe
df = pd.DataFrame(data_list)

new_data=[year,month,date]

# Make predictions using the loaded model
predictions = loaded_model.predict([new_data])

# Interpret the results
print(predictions)

Enter the year:2023
Enter the Month:5
Enter the date:14
[0]


/usr/local/lib/python3.8/dist-packages/sklearn/base.py:420: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
